In [2]:

import pennylane as qml
from pennylane import numpy as np 
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [3]:


# Load Breast Cancer Dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Standardize Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-Test Split (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Function to Normalize Data for Amplitude Encoding
def normalize_features(features):
    padded_length = 2 ** np.ceil(np.log2(features.shape[1])).astype(int)  # Pad to nearest power of 2
    padded_features = np.zeros((features.shape[0], padded_length))
    padded_features[:, :features.shape[1]] = features
    return padded_features / np.linalg.norm(padded_features, axis=1, keepdims=True)

# Normalize Train and Test Features
X_train_norm = normalize_features(X_train)
X_test_norm = normalize_features(X_test)

# Quantum Circuit Setup
n_qubits = int(np.log2(X_train_norm.shape[1]))  # log2(number of features)
dev = qml.device("default.qubit", wires=n_qubits)

def quantum_circuit(weights, x):
    """Amplitude encoding and a variational ansatz."""
    qml.AmplitudeEmbedding(features=x, wires=range(n_qubits), normalize=False)
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))

# Initialize weights for variational circuit
num_layers = 6  # Increased the number of layers
weights_shape = (num_layers, n_qubits, 3)  # StronglyEntanglingLayers requires 3 parameters per qubit
weights = np.random.randn(*weights_shape, requires_grad=True)

@qml.qnode(dev)
def circuit(weights, x):
    return quantum_circuit(weights, x)

# Define Cost Function
def cost(weights, X, y):
    predictions = np.array([circuit(weights, x) for x in X])
    return np.mean((predictions - y) ** 2)

# Train Quantum Model
opt = qml.AdamOptimizer(stepsize=0.1)  # Using Adam optimizer
steps = 100

for step in range(steps):
    weights, cost_value = opt.step_and_cost(lambda w: cost(w, X_train_norm, y_train), weights)
    if step % 10 == 0:
        print(f"Step {step}: Cost = {cost_value}")

# Evaluate Quantum Model
y_pred_quantum = [1 if circuit(weights, x) > 0.5 else 0 for x in X_test_norm]
quantum_accuracy = accuracy_score(y_test, y_pred_quantum)
print(f"Quantum Classifier Accuracy: {quantum_accuracy}")
print(classification_report(y_test, y_pred_quantum))

# Train Logistic Regression Model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Evaluate Logistic Regression Model
y_pred_logistic = log_reg.predict(X_test)
logistic_accuracy = accuracy_score(y_test, y_pred_logistic)
print(f"Logistic Regression Accuracy: {logistic_accuracy}")
print(classification_report(y_test, y_pred_logistic))


Step 0: Cost = 0.5906784417542017
Step 10: Cost = 0.2358198881991129
Step 20: Cost = 0.20716124116004583
Step 30: Cost = 0.19418780576695824
Step 40: Cost = 0.1860523558563575
Step 50: Cost = 0.17890193831890366
Step 60: Cost = 0.1762818126432812
Step 70: Cost = 0.1728142516839597
Step 80: Cost = 0.16974593279643863
Step 90: Cost = 0.16889685027849446
Quantum Classifier Accuracy: 0.695906432748538
              precision    recall  f1-score   support

           0       0.59      0.57      0.58        63
           1       0.75      0.77      0.76       108

    accuracy                           0.70       171
   macro avg       0.67      0.67      0.67       171
weighted avg       0.69      0.70      0.69       171

Logistic Regression Accuracy: 0.9824561403508771
              precision    recall  f1-score   support

           0       0.97      0.98      0.98        63
           1       0.99      0.98      0.99       108

    accuracy                           0.98       171
   ma

In [4]:
# Increase Circuit Depth
num_layers = 12  # Doubling the number of layers

# Update weights initialization
weights_shape = (num_layers, n_qubits, 3)  # StronglyEntanglingLayers requires 3 parameters per qubit
weights = np.random.randn(*weights_shape, requires_grad=True)

# Re-train Quantum Model
opt = qml.AdamOptimizer(stepsize=0.1)  # Using Adam optimizer
steps = 100

for step in range(steps):
    weights, cost_value = opt.step_and_cost(lambda w: cost(w, X_train_norm, y_train), weights)
    if step % 10 == 0:
        print(f"Step {step}: Cost = {cost_value}")

# Evaluate Quantum Model
y_pred_quantum = [1 if circuit(weights, x) > 0.5 else 0 for x in X_test_norm]
quantum_accuracy = accuracy_score(y_test, y_pred_quantum)
print(f"Quantum Classifier Accuracy: {quantum_accuracy}")
print(classification_report(y_test, y_pred_quantum))


Step 0: Cost = 0.5429875182369339
Step 10: Cost = 0.19897933862760545
Step 20: Cost = 0.17096259646103631
Step 30: Cost = 0.15996260061925302
Step 40: Cost = 0.1533412454879959
Step 50: Cost = 0.14819300855716713
Step 60: Cost = 0.14553318502894672
Step 70: Cost = 0.14333804933295616
Step 80: Cost = 0.14149475875769119
Step 90: Cost = 0.13947249096765504
Quantum Classifier Accuracy: 0.7368421052631579
              precision    recall  f1-score   support

           0       0.70      0.51      0.59        63
           1       0.75      0.87      0.81       108

    accuracy                           0.74       171
   macro avg       0.72      0.69      0.70       171
weighted avg       0.73      0.74      0.73       171

